<a href="https://colab.research.google.com/github/SimNaeun/Google_Bootcamp/blob/main/LLM_Science_Exam_Ranked_Predictions_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 훈련데이터 불러오기

In [ ]:
!pip install kaggle

In [ ]:
#kaggle.json 파일 불러오기
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"snaeun","key":"c04c44621f2fcba7568fe3e146ec700d"}'}

In [ ]:

!mkdir -p ~/.kaggle #kaggle 폴더 생성
!cp kaggle.json ~/.kaggle/ # kaggle.json 파일을 새로 생성한 kaggle 폴더에 붙여넣기
!chmod 600 ~/.kaggle/kaggle.json #소유자에게만 읽기 쓰기에 대한 파일 권한 부여


In [ ]:
!kaggle competitions download -c kaggle-llm-science-exam

kaggle-llm-science-exam.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip kaggle-llm-science-exam.zip

Archive:  kaggle-llm-science-exam.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd

train_df = pd.read_csv('/content/train.csv')
train_df.head()

,id,prompt,A,B,C,D,E,answer
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


In [ ]:

!pip install datasets

In [ ]:
#데이터 프레임을 데이터셋으로 변환
from datasets import Dataset
train_ds = Dataset.from_pandas(train_df)

# 전처리 및 토크나이징

In [ ]:
pip install transformers

In [ ]:
from transformers import AutoTokenizer

#사용하고자 하는 모델 체크포인트의 경로
model_dir = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_dir)

In [ ]:
# A, B, C, D, E(option) 를 indices로 변환하고 다시 불러오기 위한 사전 생성
options = 'ABCDE'
indices = list(range(5))

option_to_index = {option: index for option, index in zip (options, indices)}
index_to_option = {index: option for option, index in zip (options, indices)}

def preprocess(example):
  # AutoModelForMultipleChoice class는 질문/답 쌍 세트
  # 토큰화하기 전 질문을 5번 복사
  first_sentence = [example['prompt']] * 5
  second_sentence = []
  for option in options:
    second_sentence.append(example[option])
  # tokenizer를 통해 BERT가 이해할 수 있는 토큰 ID로 text를 변환
  tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True)
  tokenized_example['label'] = option_to_index[example['answer']]
  return tokenized_example

tokenized_train_ds = train_ds.map(preprocess, batched = False, remove_columns= ['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

# DataCollatorForMultipleChoice: Hugging Face의 Transformers 라이브러리를 사용해 다중선택 태스크를 위한 데이터 처리 수행
# 데이터 전처리, 모델 학습에 사용하기 위한 데이터 형식으로 변환
@dataclass # '데코레이터'는 데이터 클래스(데이터 저장 및 초기화를 위한 메서드를 자동으로 생성)
#('__init__' 메서드)의 매개변수 ex. tokenizer, padding...
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase # 사전훈련된 토크나이저의 예시
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None # 최대 시퀀스 길이 지정
    pad_to_multiple_of: Optional[int] = None # 패딩 길이를 특정 값의 배수로 지정
#'__call__'  메서드: 입력 데이터를 처리하고 다중 선택 태스크에 맞게 준비
# 입력 데이터: 각각의 딕셔너리는 여러 선택지를 가진 질문 세트(리스트)
# 각 딕셔너리에서 정답 선택지 인덱스인 레이블 추출 및 입력 기능에서 제거
# 입력 기능 평면화 -> 모든 질문에 대한 단일 선택지에 해당하는 딕셔너리 목록으로 변환
# 토크나이저를 사용해 평면화된 기능 패딩, PyTorch 텐서로 변환
# 텐서 모양 재구성, 레이블을 별도의 텐서로 추가
    def __call__(self, features):
        label_name = "label" if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch


# Training

In [ ]:
!pip install -U huggingface_hub

In [ ]:
#다중 선택 태스크를 위한 모델을 불러오고 초기화
#AutoModelForMultipleChoice: 다중 선택 태스크를 위한 자동 모델 클래스
#TrainingArguments: 하이퍼파라미터 등을 설정
#Trainer: 모델 학습 관리 및 실행
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

#사전훈련된 다중 선택 모델 로드
# model_dir: 사전 훈련된 모델 이름
# 모델 인스턴스 생성: 로드된 모델을 model에 할당
model = AutoModelForMultipleChoice.from_pretrained(model_dir)

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
pip install accelerate -U

In [ ]:
# 모델이 학습 중 저장될 때, 학습이 완료된 후 모델 체크포인트와 결과 저장
model_dir = 'finetuned_bert'

#trainingArguments: 하이퍼파라미터 등을 설정
training_args = TrainingArguments(
    output_dir = model_dir, # 모델 체크포인트와 학습 결과 저장할 디렉토리 경로
    evaluation_strategy = "epoch", # 평가전략: 각 에코크의 끝에서 평가 수행
    save_strategy = "epoch", # 모델 저장 전략, 각 에포크의 끝에서 모델 체크포인트 저장
    load_best_model_at_end = True, # 학습 종료 시 가장 좋은 성능을 보인 모델 불러옴
    learning_rate = 5e-5,
    per_device_train_batch_size = 4, # 학습을 수행하는 데 device(ex. GPU)가 여러개인 경우 각 device에 동시에 몇 개의 데이터 샘플을 보낼 것인지 지정
    per_device_eval_batch_size = 4,
    num_train_epochs = 3,
    weight_decay = 0.01, #가중치 정규화를 통해 과적합 방지
    report_to = 'none' # 학습 결과를 어떤 곳에도 보고하지 않음
)

In [ ]:
import numpy as np
# 모델의 예측값을 가공하여 특정 형식의 출력으로 변환
def predictions_to_map_output(predictions):
  sorted_answer_indices = np.argsort(-predictions) # predictions 배열에서 각 예측의 순위를 계산해 내림차순으로 정렬
  top_answer_indices = sorted_answer_indices[:,:3] # 각 입력에 대해 상위 3개의 예측값 인덱스 선택
  top_answers = np.vectorize(index_to_option.get)(top_answer_indices)
  #예측값을 실제 답변으로 변환, index_to_option 딕셔너리를 사용해 인덱스를 실제 답변으로 매핑
  #np.vectorize를 사용해 함수 벡터화 및 배열 요소에 대한 매핑 수행
  return np.apply_along_axis(lambda row: ' '.join(row), 1, top_answers)
  #top_answers 배열의 각 행에 있는 상위3개의 예측값을 공백으로 구분된 문자열로 결합
  #np.apply_along_axis: 각 행에 대해 이 작업 수행

In [ ]:
def competition_score(y_true, y_pred):
    ap_at_3 = 0.0 # 정답이 맞는 경우의 개수 추적
    for i in range(len(y_true)):
        if y_true[i] in y_pred[i][:3]: # 실제 정답이 상위 3개 예측값 중에 포함되어 있는가
            ap_at_3 += 1 #포함되어 있으면 1증가
    map3 = ap_at_3 / len(y_true) # 상위 3개 예측값이 정확한 비율
    return map3
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    y_pred = predictions_to_map_output(logits) # 모델의 출력을 경쟁 점수 계산에 사용할 수 있는 형식으로 변환
    y_true = [index_to_option[label] for label in labels] # 각 예제의 실제 정답을 가져옴
    return {'Map@3': np.round(competition_score(y_true, y_pred), 3)} # MAP@3 계산

In [ ]:
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_train_ds,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics
    #데이터 처리에 사용할 data collator 지정, 입력 데이터를 모델에 맞게 준비, 토큰화된 데이터를 배치로 만들고 패딩 추가 등의 작업 수행
)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Map@3
1,No log,1.511820,0.770000
2,No log,1.186546,0.815000
3,No log,1.060555,0.910000


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=150, training_loss=1.4079373168945313, metrics={'train_runtime': 76.2082, 'train_samples_per_second': 7.873, 'train_steps_per_second': 1.968, 'total_flos': 178071141202320.0, 'train_loss': 1.4079373168945313, 'epoch': 3.0})

# Prediction

In [ ]:
predictions = trainer.predict(tokenized_train_ds)

In [ ]:
predictions_to_map_output(predictions.predictions)

array(['D B A', 'A D C', 'A C D', 'C A D', 'D A E', 'B C A', 'D B A',
       'D B E', 'C E A', 'A C E', 'C E B', 'A D B', 'C B A', 'E B D',
       'B A E', 'E D C', 'E B C', 'E A B', 'A D B', 'B E D', 'B D C',
       'E D B', 'C B D', 'C A B', 'E A B', 'E B A', 'A C E', 'D C B',
       'E B A', 'C E B', 'B D C', 'E A C', 'E D B', 'D A E', 'E D C',
       'D B A', 'E B A', 'A D C', 'E D B', 'C A E', 'E D B', 'C A E',
       'B C D', 'D C B', 'A B D', 'A B C', 'B E A', 'C E A', 'B C E',
       'B A E', 'B D C', 'E B A', 'C A E', 'A C D', 'B A C', 'B E C',
       'C A B', 'C B A', 'A D E', 'B A C', 'B E D', 'B C E', 'C B A',
       'C A E', 'A B D', 'E C D', 'C D E', 'E C B', 'E C D', 'D E C',
       'B C A', 'A E D', 'C A B', 'E B D', 'D C A', 'C B E', 'A B C',
       'B E D', 'C B D', 'E A D', 'C D E', 'C B A', 'D B E', 'A C B',
       'C E B', 'B D C', 'D E B', 'D B A', 'E C A', 'D B A', 'D B A',
       'B D C', 'B E C', 'E D A', 'E B D', 'C B A', 'C D A', 'D E B',
       'C B D', 'E D

In [ ]:
test_df = pd.read_csv('/content/test.csv')
test_df.head()

,id,prompt,A,B,C,D,E
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...


In [ ]:
#test_df에 'asnwer'라는 열 추가하고 모든 행의 값을 'A'로 설정=모든 입력에 대해 'A'라는 옵션 예측하도록
test_df['answer']='A'

test_ds = Dataset.from_pandas(test_df)
#test_Df를 dataset 객체로 변환

tokeninzed_test_ds = test_ds.map(preprocess, batched=False, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
#map 메서드를 사용해 데이터셋을 한번에 텍스트 처리 및 서로 다른 길이의 시퀀스 패딩 및 잘라내기
#preprocess: 데이터 토큰화 및 다른 전처리 작업 수행
#batched=False: batch 처리를 사용하지 않고 예제를 하나씩 처리
#['prompt', 'A', 'B', 'C', 'D', 'E', 'answer']: 모델의 입력과 관련없는 열('answer'열에 저장되었기에 제거)



Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
test_predictions = trainer.predict(tokeninzed_test_ds)

In [ ]:
submission_df = test_df[['id']]
#id 열은 각 입력 샘플을 식별하기 위한 고유한 식별자
submission_df['prediction'] = predictions_to_map_output(test_predictions.predictions)
#모델의 예측값을 가공 및 prediction 열에 추가
#예측값은 입력에 대한 상위 3개의 예측옵션을 공백으로 구분된 문자열로 나타냄

submission_df.head()

<ipython-input-28-5359c889ce05>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_df['prediction'] = predictions_to_map_output(test_predictions.predictions)


,id,prediction
0,0,D B A
1,1,A D C
2,2,A C D
3,3,C A D
4,4,D A E
